# Inductive Node classification using GraphSAGE
---  

In this tutorial, we will use `Scikit-Network` [1] implementation of $\texttt{GraphSAGE}$ [2], a Graph Neural Network model, to solve a node classification task on [wikivitals dataset](https://netset.telecom-paris.fr/pages/wikivitals.html).  

The visualizations in this tutorial are built using [UMAP](https://umap-learn.readthedocs.io/en/latest/) [3] and [Plotly](https://plotly.com/graphing-libraries/).

References:
* [1] T. Bonald, N. de Lara, Q. Lutz, B. Charpentier (2020), [Scikit-network: Graph Analysis in Python](https://www.jmlr.org/papers/volume21/20-412/20-412.pdf)
* [2] Hamilton, W. Ying, R., & Leskovec, J. (2017), [Inductive Representation Learning on Large Graphs.](https://arxiv.org/pdf/1706.02216.pdf)
* [3] L. McInnes, J. Healy, J. Melville (2018), [Umap: Uniform manifold approximation and projection for dimension reduction.](https://arxiv.org/pdf/1802.03426.pdf?source=post_page---------------------------)

**Install and imports**

In [ ]:
!pip install numpy==1.22
!pip install scikit-network

In [ ]:
from IPython.display import SVG

import numpy as np
from scipy import sparse
import warnings
warnings.filterwarnings("ignore")

from sknetwork.classification import get_accuracy_score
from sknetwork.data import load_netset
from sknetwork.gnn import GNNClassifier
from sknetwork.linalg.normalization import normalize
from sknetwork.ranking.postprocess import top_k
from sknetwork.utils import get_degrees
from sknetwork.visualization import svg_graph

# 1. Introduction

### 1.1 GraphSAGE

Recently, deep learning methods showed significant performance in prediction tasks in numerous research fields, including computer vision and natural language processing. **Graph Neural Network (GNN)** approaches have been specifically designed to take advantage of these great performances while applying them to graphs.  
One of these methods, $\texttt{GraphSAGE}$, has been created to generalize well to **unseen nodes**, in an *inductive* manner. The approach relies on a context-based similarity assumption, which states that nodes in the same neighborhood are similar and thus should be close in the embedding space. In addition to the aggregation scheme developed in previous GNN methods, $\texttt{GraphSAGE}$ also includes a **sampling** module which allows to reduce the computation time of the learning process. 

The embedding generation process for a graph $\mathcal{G}=(\mathcal{V}, \mathcal{E})$ works as follows: at each layer $k$, we (i) aggregate information from node's $v \in \mathcal{V}$ neighborhood $\mathcal{N}(v)$, using $\texttt{AGGREGATE}$ function (e.g mean) to produce the neighborhood representation of node's $v$, $h^{k}_{\mathcal{N}(v)}$ and (ii) use $\texttt{CONCAT}$ function (e.g concatenation or sum in practice) on both the neighborhood representation $h^{k}_{\mathcal{N}(v)}$ and the node's previous representation $h^{k-1}_v$ to produce the final embedding of node $v$, $h^{k}_v$.

$$
h^{k}_{\mathcal{N}(v)} \leftarrow \text{AGGREGATE}_k(\{h_{u}^{k-1}, \forall{u} \in \mathcal{N}(v)\}) \\
h^{k}_{v} \leftarrow \sigma \biggl(W^{k} \cdot \text{CONCAT}\big(h^{k-1}_v, h^{k}_{\mathcal{N}(v)}\big) \biggl)
$$

Such a model can be trained specifically on a downstream machine learning task, such as node classification, by using a loss function which enforces similar nodes to be close in the embedding space, and non-similar nodes to be set further appart.  

Note: Different aggregators are used in [2] (`mean`, `LSTM`, `MaxPooling`). In this tutorial, we will use the `mean` aggregator.

### 1.2 Scikit-network and sparse formats  

Scikit-network is a Python package for the analysis of large graphs. For this purpose, it uses memory-efficient representation of graphs as sparse matrices in [CSR](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)  (Compressed Sparse Row) format.  
Let's see how this format works.

In [ ]:
# Random matrix (dense format)
X_dense = np.random.randint(3, size = (10,5))
X_dense

In [ ]:
# Convert to sparse CSR format
X_csr = sparse.csr_matrix(X_dense)
X_csr

Our matrix is now in CSR format. We can get useful information about it using specific parameters.

In [ ]:
X_csr.shape

In [ ]:
# Number of non-zero values
X_csr.nnz

Our matrix is stored as 3 arrays accessible as parameters of the CSR matrix: 
* `indices`: array of column indices
* `data`: array of corresponding non-zero values
* `indptr`: points to row starts in `indices` and `data`

In [ ]:
X_csr.indices

In [ ]:
X_csr.indptr

In [ ]:
X_csr.data

# 2. Node classification

In node classification task, the goal is to predict the categorical label of nodes in a graph. For this purpose, we can use deep-learning based graph models, such as $\texttt{GraphSAGE}$, to convert high-dimensional objects into low-dimensional latent space, then use these learned representations to predict the belonging class of each node.

## 2.1 Data: `wikivitals`

[Wikivitals](https://netset.telecom-paris.fr/pages/wikivitals.html) contains Wikipedia vitals articles. In this dataset, each node is a Wikipedia article and a directed edge exists between two nodes if one article is citing the other. Each node comes with a feature vector which consists of the words used in its summary, as well as a category (e.g 'Mathematics', 'People', etc.). Features are represented in a bag-of-words format, where the $i^{th}$ element of the feature vector contains the number of occurences of the $i^{th}$ word of a vocabulary in the article.

In summary, `wikivitals` dataset contains:
* 10011 articles, with 824999 links between them
* 37845 distinct words and 1363301 links between articles and words
* 11 distinct labels 

The goal is to predict the category of each node in the graph, using both the structural and feature information we got.

In [ ]:
# Load data
graph = load_netset('wikivitals')
adjacency = graph.adjacency # graph of articles
features = graph.biadjacency # graph of articles*words
names = graph.names # article names
names_features = graph.names_col # word names
names_labels = graph.names_labels # category names
labels = graph.labels # categories

In [ ]:
# Names of labels to predict
names_labels

In [ ]:
# Number of distincty labels
n_labels = len(names_labels)
n_labels

In [ ]:
# Adjacency matrix of the graph, i.e graph of articles
adjacency

In [ ]:
# 10 random article names
np.random.choice(names, 10)

In [ ]:
# Features of the nodes, i.e bags-of-words
features

In [ ]:
# Information about a specific article
i = np.random.choice(adjacency.shape[0])

print(f'Article: {names[i]} - Category: {names_labels[labels[i]]}')
words = names_features[features[i].indices]

if len(words) > 15:
    words = np.random.choice(words, 15)
print(f'Random words: {words}')

## 2.2 Model: GraphSAGE

### 2.2.1 Predictions without training  

We will use the `GNNClassifier` object to initialize our model using $\texttt{GraphSAGE}$ layers. For this purpose, we define:
* dimensions for hidden and output layer
* layer type 
* activation functions 
* sample sizes parameters

In [ ]:
# GNN classifier with a single hidden layer
hidden_dim = 32

gnn = GNNClassifier(dims=[hidden_dim, n_labels],
                    layer_types='graphsage',
                    activations=['Relu', 'Softmax'],
                    sample_sizes=[25, 10],
                    verbose=True)

In [ ]:
print(gnn)

Before training the model, let's use it as an encoder (without training) to create embeddings for each node in our graph. We use the `forward()` method with adjacency and feature matrices.

In [ ]:
# Weights randomly initialized - no training
output_no_training = gnn.forward(adjacency, features)

# Predictions - no training
labels_no_training = np.argmax(output_no_training, axis=1)

In [ ]:
# Accuracy
accuracy_without_training = get_accuracy_score(labels, labels_no_training)
print(f'Accuracy without training: {accuracy_without_training:.4f}')

What happens under the hood?  
* Shapes of parameters $W$ and $b$
* Shape of embedding and outputs for each layer

In [ ]:
for i, l in enumerate(gnn.layers):
    print(f'Layer {i} | Weights: {l.weight.shape} - bias: {l.bias.shape} - embedding: {l.embedding.shape} - output: {l.output.shape}')

**Visualization**

In [ ]:
!pip install pandas scikit-learn numba tqdm pynndescent matplotlib datashader holoviews 
!pip install umap-learn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import umap
import umap.plot

In [ ]:
# Static 2d visualization
mapper = umap.UMAP().fit(gnn.layers[1].embedding)
umap.plot.points(mapper, labels=names_labels[labels]);

### 2.2.2 Predictions with supervised learning  

We can now train our GNN model using the previously initialized object. We split our data using train/val/test subgraphs.

In [ ]:
labels_pred = gnn.fit_predict(adjacency, features, labels, 
                              train_size=0.8, val_size=0.1, test_size=0.1, 
                              history=True)

Now that the GNN model is trained, we have access to the full history of training information saved in `history_` parameter, as well as model a summary of results stored in parameters:
* `embedding_`: last nodes embedding
* `output_`: predicted probabilities for each class
* `labels_`: predicted node labels

In [ ]:
gnn.history_.keys()

In [ ]:
print(f'Node embedding: {gnn.embedding_.shape}')
gnn.embedding_[0, :]

In [ ]:
# Predicted probabilities for each class
print(gnn.output_[1, :])

In [ ]:
# Predicted labels
gnn.labels_

In [ ]:
# plot learning information
fig, ax = plt.subplots(1, 2, figsize=(7, 4))
ax[0].plot(gnn.history_.get('loss'), label='Loss')
ax[1].plot(gnn.history_.get('train_accuracy'), label='train accuracy')
ax[1].plot(gnn.history_.get('val_accuracy'), label='val accuracy')
for i in range(2):
    ax[i].set_xlabel('# epochs')
    ax[i].legend()

In [ ]:
# Accuracy on test set
test_mask = gnn.test_mask
get_accuracy_score(labels[test_mask], labels_pred[test_mask])

In [ ]:
# Explore mis-predicted nodes
mispreds = labels != labels_pred

for i in names[mispreds][-5:]:
    idx = np.where(names == i)[0][0]
    print(f'{i}')
    print(f'    True label: {names_labels[labels[idx]]:} - predicted label: {names_labels[labels_pred[idx]]}')

**Todo:**
* List the 20 closest articles to the category **'Mathematics'** in terms of cosine similarity in the embedding space.

In [ ]:
# Todo

**Visualization**  

We use [UMAP library](https://umap-learn.readthedocs.io/en/latest/) (Uniform Manifold Approximation and Projection for Dimension Reduction) and [Plotly]() to plot our results.

In [ ]:
!pip install plotly

In [ ]:
import plotly
from plotly import offline
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# Utility functions
   
def build_df(graph, embedding, labels_pred, node_type='article', mask=None):
    """ Build DataFrame from graph data, node embedding and predicted labels """
    
    n = graph.adjacency.shape[0]
    
    # Select 5 features randomly
    words_sel = [graph.names_col[np.random.choice(graph.biadjacency[i].indices, 5)] for i in range(n)]

    # Get coordinates and attributes
    df_coord = pd.DataFrame(embedding, columns=['X', 'Y', 'Z'])
    if node_type == 'article':
        df_qual = pd.DataFrame({'Article': graph.names, 
                                'Label': graph.names_labels[graph.labels], 
                                'Predicted label': graph.names_labels[labels_pred],
                                'Words': words_sel})
        df = pd.concat((df_coord, df_qual), axis=1)

        # Size of marker according to accuracy of category
        unique_labels, counts = np.unique(graph.labels, return_counts=True)
        for lab, c in zip(unique_labels, counts):
            acc = np.sum(labels_pred[graph.labels == lab] == lab) / c
            df.loc[df['Label'] == graph.names_labels[lab], 'group acc'] = round(acc, 3)
    elif node_type == 'word':
        df_qual = pd.DataFrame({'Word': graph.names_col[mask], 
                                'Predicted label': graph.names_labels[labels_pred]})
        df = pd.concat((df_coord, df_qual), axis=1)
    
    return df

In [ ]:
# Static 2d visualization
mapper = umap.UMAP(n_components=2).fit(gnn.embedding_)
umap.plot.points(mapper, labels=names_labels[labels]);

In [ ]:
# 3d interactive using Plotly
size = 800

# Dimension reduction of GNN embedding using UMAP
labels_unique, counts = np.unique(labels, return_counts=True)
umap_emb = umap.UMAP(n_components=3).fit_transform(gnn.embedding_)

df = build_df(graph, umap_emb, labels_pred)
fig = px.scatter_3d(df, x='X', y='Y', z='Z', opacity=0.6, color='Label', 
                    hover_data={'X': False, 'Y':False, 'Z': False, 'group acc': True, 
                                'Article':True, 'Label': True, 'Predicted label': True, 'Words': True}, 
                    width=size*1.2, height=size, color_discrete_sequence=px.colors.qualitative.G10)

fig.update_traces(marker=dict(size=3, line=dict(width=0)))
fig.show()
#plotly.offline.plot(fig, filename='wikivitals_3d_embedding.html')

### 2.2.3 What if we remove features?  

We can try to learn node embeddings by setting **random features** and keep only the graph structure as input, in order to get a deeper insight of their importance in the learning process.

In [ ]:
# Initialize random features
rand_features = sparse.random(features.shape[0], features.shape[1]).tocsr()
rand_features

In [ ]:
# GNN classifier with a single hidden layer
hidden_dim = 16

gnn_rand_feat = GNNClassifier(dims=[hidden_dim, n_labels],
                    layer_types='graphsage',
                    activations=['Relu', 'Softmax'],
                    sample_sizes=[25, 10],
                    verbose=True)

labels_pred_rand_feat = gnn_rand_feat.fit_predict(adjacency, rand_features, labels, 
                              train_size=0.8, val_size=0.1, test_size=0.1, 
                              history=True)

In [ ]:
# Accuracy on test set
test_mask = gnn_rand_feat.test_mask
get_accuracy_score(labels[test_mask], labels_pred_rand_feat[test_mask])

### 2.2.4 What if we remove links between articles?  

In the same manner, we can remove all edges between articles and train the model only using features to understand the importance of the structure of the graph.

In [ ]:
# Adjacency matrix is the identity matrix
identity_adjacency = sparse.identity(adjacency.shape[0]).tocsr()

In [ ]:
# GNN classifier with a single hidden layer
hidden_dim = 16


gnn_id_adj = GNNClassifier(dims=[hidden_dim, n_labels],
                    layer_types='graphsage',
                    activations=['Relu', 'Softmax'],
                    sample_sizes=[25, 10],
                    verbose=True)

labels_pred_id_adj = gnn_id_adj.fit_predict(identity_adjacency, features, labels, 
                              train_size=0.8, val_size=0.1, test_size=0.1, 
                              history=True)

In [ ]:
# Accuracy on test set
test_mask = gnn_id_adj.test_mask
get_accuracy_score(labels[test_mask], labels_pred_id_adj[test_mask])

# 3 Inductive predictions

## 3.1 New article 

In [ ]:
# Utility function

def plot_new_article(graph, df, labels_pred, categories):
    """Plot article embedding, with new article highlighted."""
    
    traces = []
    colors = px.colors.qualitative.Alphabet 

    df['norm op'] = df['group acc'].apply(
                        lambda x: (x - np.min(df['group acc'])) / (np.max(df['group acc']) - np.min(df['group acc'])))

    n_labels = len(graph.names_labels)
    for lab in range(n_labels):
        sub_df = df[df['Label']==graph.names_labels[lab]]
        traces.append(go.Scatter3d(
                x = sub_df.X,
                y = sub_df.Y,
                z = sub_df.Z,
                name = graph.names_labels[lab],
                mode = 'markers',
                opacity = np.clip(np.array(sub_df['norm op'].values[0]), 0.1, 0.9), 
                marker = dict(size=3, line=dict(width=0), color=colors[lab]),
                text = sub_df['Article'],
                customdata = sub_df['Predicted label'],
                hovertemplate =
                    "<br>Article: %{text}<br>" +
                    f"<br>Label: {graph.names_labels[lab]}<br>" +
                    "<br>Pred label: %{customdata}<br>" + 
                    f"<br>Category acc: {sub_df['group acc'].unique()[0]}<br>"
                    "<extra></extra>"
                )
        )
    traces.append(go.Scatter3d(
        x = [df.iloc[-1].X],
        y = [df.iloc[-1].Y],
        z = [df.iloc[-1].Z],
        name = 'New article',
        marker = dict(size=10, line=dict(width=1)),
        customdata = graph.names_labels[labels_pred],
        hovertemplate =
                    "<br>Article: New Article<br>" +
                    f"<br>Label: {graph.names_labels[categories]}<br>" +
                    "<br>Pred label: %{customdata}<br>"
                    "<extra></extra>"

    ))

    layout = go.Layout(
        title='Wikivitals embedding',
        width=1000,
        height=800,
        autosize=False,
    )

    fig = dict(data=traces, layout=layout)

    offline.iplot(fig)

### 3.1.1 Using only words

We can use our pre-trained GNN to predict the category of a new incoming article, using only the words in its summary. For this, we create a new article without any link to existing articles. Then, we create the summary of this article by selecting random words in dedicated categories. Our goal is to recover the category of the node, by feeding it to the forward method of our model.

In [ ]:
# Select category
category_idx = [10]
print(f'Category: {names_labels[category_idx]}')

In [ ]:
keywords = []

# Select 50 random words from articles in category
for lab_idx in category_idx:
    mask = labels == lab_idx
    words_idx = np.flatnonzero(get_degrees(features[mask, :], transpose=True))
    keywords_lab = np.random.choice(words_idx, int(np.round(50 / len(category_idx))))
    keywords += list(keywords_lab)
    
keywords = np.array(keywords)
print(f'Words in new articles: {names_features[keywords]}')

In [ ]:
# Build new article using selected words
new_features = np.zeros(features.shape[1], dtype=bool)
new_features[keywords] = 1

In [ ]:
# Predict label of new article
labels_pred_new_article = gnn.predict(None, new_features.reshape(1, -1))

In [ ]:
print(f'Predicted category for new article: {names_labels[labels_pred_new_article]}')

In [ ]:
# Most similar articles
norm_embedding = normalize(gnn.embedding_, p=2)
new_norm_embedding = normalize(gnn.layers[-1].embedding, p=2) 

similarities = norm_embedding.dot(new_norm_embedding.T)
print(names[top_k(similarities.ravel(), 10)])

**Visualize new article embedding**

In [ ]:
# UMAP embedding including new article
new_emb = np.concatenate((gnn.embedding_, gnn.layers[-1].embedding), axis=0)
new_umap_emb = umap.UMAP(n_components=3).fit_transform(new_emb)

# DataFrame
new_df = build_df(graph, new_umap_emb, labels_pred)

In [ ]:
plot_new_article(graph, new_df, labels_pred_new_article, category_idx)

### 3.1.2 Using only references

Finally, we can create a new **empty article**, i.e an article without any word in its summary, by defining only its neighborhood through the relations with other similar articles. Again, the goal is to predict the category of the article using our pre-trained model.

In [ ]:
category_idx = 8
n_nodes = 10 # number of neighbors for our new empty article

In [ ]:
# Select n_nodes nodes in a category
rand_articles = np.random.choice(np.flatnonzero([labels == category_idx]), n_nodes)
print(f'Selected articles in category {names_labels[category_idx]}: {names[rand_articles]}')

In [ ]:
n = adjacency.shape[1]

# Connect empty article with its neighbors
adjacency_vector = sparse.csr_matrix((np.ones(n_nodes), (np.zeros(n_nodes), rand_articles)), 
                                     shape=(1, n))

# new adjacency (with outgoing links from the new node)
adjacency_new = sparse.vstack((adjacency, adjacency_vector))
adjacency_new = sparse.hstack((adjacency_new, sparse.csr_matrix((n + 1, 1)))).tocsr()

# new features (null features for the new node)
features_new = sparse.vstack((features, sparse.csr_matrix((1, len(names_features)))))

In [ ]:
# Filter adjacency on new article and neighbors
adjacency_vector_neighbs = np.concatenate((rand_articles, np.array([n])))
adjacency_neighbs = adjacency_new[adjacency_vector_neighbs, :][:, adjacency_vector_neighbs]

In [ ]:
# Visualize our new empty article and its neighborhood
SVG(svg_graph(adjacency_neighbs, 
              names=np.hstack((names[rand_articles], np.array(['Empty article']))), 
              labels=np.hstack((labels[rand_articles], np.array([n_labels + 1])))))

In [ ]:
# Label prediction
labels_new = gnn.predict(adjacency_new, features_new)

print(f'Label of neighbor nodes: {names_labels[category_idx]}')
print(f'Predicted label: {names_labels[labels_new[-1]]}')

In [ ]:
# UMAP embedding including new article
new_emb = np.concatenate((gnn.embedding_, gnn.layers[-1].embedding), axis=0)
new_umap_emb = umap.UMAP(n_components=3).fit_transform(new_emb)

# DataFrame
new_df = build_df(graph, new_umap_emb, labels_pred)

In [ ]:
plot_new_article(graph, new_df, [labels_new[-1]], category_idx)